In [3]:
import os
import mdtraj as md
from pymol import cmd

# 文件路径配置
trajectory_file = "/scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Outputs/metad1_initmonitor/METADYN-pos-00.xyz.gz"
topology_file = "/scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Outputs/metad1_initmonitor/complex_LJ_mod.prmtop"
pdb_movies_file = "/scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Analysis/trajectorymovies.pdb"
output_dir = "/scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures"
frame_dir = os.path.join(output_dir, "moviesfigure")  # 存储每帧图片的目录
movie_file = os.path.join(output_dir, "reaction_movie.mp4")  # 生成的MP4电影文件
gif_file = os.path.join(output_dir, "reaction_movie.gif")  # 生成的GIF文件

# 确保输出目录存在
os.makedirs(frame_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 重新生成未抽帧的 PDB 文件
try:
    print("Loading trajectory and topology files...")
    traj = md.load(trajectory_file, top=topology_file)
    print("Saving full-frame trajectory to PDB file...")
    traj.save_pdb(pdb_movies_file)
    print(f"Full-frame PDB trajectory successfully saved at: {pdb_movies_file}")
except Exception as e:
    print(f"Error while generating full-frame PDB: {e}")

# 使用 PyMOL 生成每帧图像
try:
    print("Loading PDB file into PyMOL...")
    cmd.reinitialize()
    cmd.load(pdb_movies_file, "traj")

    # 设置显示样式
    cmd.hide("everything")
    cmd.show("cartoon", "all")
    cmd.select("ligand", "resname CHA")
    cmd.select("nearby", "byres (ligand around 3)")
    cmd.show("sticks", "ligand or nearby")
    cmd.color("cyan", "ligand and elem C")
    cmd.color("magenta", "nearby and elem C")
    cmd.color("red", "elem O")
    cmd.color("blue", "elem N")
    cmd.color("yellow", "elem S")
    cmd.color("orange", "elem P")

    # 放大并居中到配体及其附近残基
    cmd.center("ligand or nearby")  # 确保视图中心在配体和附近残基
    cmd.zoom("ligand or nearby", buffer=5)  # 放大视图，缓冲范围设置为 5 Å
    cmd.bg_color("white")

    # 逐帧生成图片
    frame_count = cmd.count_states("traj")
    print(f"Generating {frame_count} frames...")
    for frame in range(1, frame_count + 1):
        cmd.frame(frame)
        cmd.ray(1920, 1080)  # 设置分辨率为 1920x1080
        frame_path = os.path.join(frame_dir, f"dynamic_bonds_movie{frame:04d}.png")
        cmd.png(frame_path, dpi=300)
        print(f"Saved frame {frame} at {frame_path}")

except Exception as e:
    print(f"Error generating frames: {e}")

# 使用 ffmpeg 生成 MP4 电影
try:
    print("Generating MP4 movie...")
    os.system(f"ffmpeg -framerate 10 -i {frame_dir}/dynamic_bonds_movie%04d.png -c:v libx264 -pix_fmt yuv420p {movie_file}")
    print(f"Movie saved at: {movie_file}")
except Exception as e:
    print(f"Error generating MP4 movie: {e}")

# 使用 ffmpeg 生成 GIF 动画
try:
    print("Generating GIF animation...")
    os.system(f"ffmpeg -framerate 10 -i {frame_dir}/dynamic_bonds_movie%04d.png -vf \"scale=800:-1:flags=lanczos\" {gif_file}")
    print(f"GIF animation saved at: {gif_file}")
except Exception as e:
    print(f"Error generating GIF animation: {e}")

Loading trajectory and topology files...
Saving full-frame trajectory to PDB file...
Full-frame PDB trajectory successfully saved at: /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Analysis/trajectorymovies.pdb
Loading PDB file into PyMOL...
Generating 16 frames...
Saved frame 1 at /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/moviesfigure/dynamic_bonds_movie0001.png
Saved frame 2 at /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/moviesfigure/dynamic_bonds_movie0002.png
Saved frame 3 at /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/moviesfigure/dynamic_bonds_movie0003.png
Saved frame 4 at /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/moviesfigure/dynamic_bonds_movie0004.png
Saved frame 5 at /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/moviesfigure/dynamic_bonds_movie0005.png
Saved frame 6 at /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week1

ffmpeg version 6.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-3)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1700971142524/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1700971142524/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1700971142524/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1700971142524/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1700971142524/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav

Movie saved at: /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/reaction_movie.mp4
Generating GIF animation...
GIF animation saved at: /scratch/wx2237/comp-lab-class/comp-lab-class-2024/Week13-QMMM/Figures/reaction_movie.gif


[out#0/gif @ 0x55c45371fa80] video:308kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.006334%
frame=   16 fps=0.0 q=-0.0 Lsize=     308kB time=00:00:01.50 bitrate=1684.3kbits/s speed=4.52x    
